In [1]:
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Grouper
from pandas import DataFrame
import csv
import pickle
import math
import calendar 

/home/jonathan/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("/home/jonathan/tesis/Data_Temperatura_Lluvia/2020_Accidentalidad.csv",sep=';')#,index_col=['fecha'], parse_dates=True)
DataAcc = df.drop(columns=['localizacion','numero','estado_meteorológico','tipo_vehiculo','tipo_persona','rango_edad','sexo','cod_lesividad','lesividad','coordenada_x_utm','coordenada_y_utm','positiva_alcohol','positiva_droga'])
DataAcc['ds'] = DataAcc['fecha'] + '/' + DataAcc['hora']
DataAcc['ds'] = pd.to_datetime(DataAcc['ds'], format='%d/%m/%Y/%H:%M:%S')
DataAcc = DataAcc.drop(columns=['fecha','hora'])
DataAcc = DataAcc.set_index('ds')
DataAcc = DataAcc.sort_index()
DataAcc

,num_expediente,cod_distrito,distrito,tipo_accidente
ds,,,,
2020-01-01 01:00:00,2020S000014,3.0,RETIRO,Colisión fronto-lateral
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000014,3.0,RETIRO,Colisión fronto-lateral
...,...,...,...,...
2020-12-31 21:55:00,2020S019513,19.0,VICÁLVARO,Atropello a persona
2020-12-31 22:00:00,2020S019516,12.0,USERA,Choque contra obstáculo fijo
2020-12-31 22:25:00,2020S019517,13.0,PUENTE DE VALLECAS,Choque contra obstáculo fijo


In [3]:
DataAcc = DataAcc.loc[((DataAcc['cod_distrito'] == 16) | (DataAcc['cod_distrito'] == 8) | (DataAcc['cod_distrito'] == 5)) # | (DataAcc['cod_distrito'] == 5) 
                       & ((DataAcc['tipo_accidente'] != 'Caída') & (DataAcc['tipo_accidente'] != 'Otro') #& (DataAcc['tipo_accidente'] != 'Vuelco')
                        & (DataAcc['tipo_accidente'] != 'Atropello a animal') & (DataAcc['tipo_accidente'] != 'Colisión frontal')
                         & (DataAcc['tipo_accidente'] != 'Solo salida de la vía'))]
DataAcc['Accidente'] = True
DataAcc = DataAcc.drop_duplicates(subset = ['num_expediente'])

/tmp/ipykernel_13019/3729481759.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataAcc['Accidente'] = True


In [4]:
# # Observar hora exacta de accidente
data1=DataAcc.loc['2020-07-06':'2020-07-06']
data1.head(60)

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente
ds,,,,,
2020-07-06 12:00:00,2020S009461,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True
2020-07-06 12:08:00,2020S009507,16.0,HORTALEZA,Colisión fronto-lateral,True
2020-07-06 17:05:00,2020S009427,8.0,FUENCARRAL-EL PARDO,Choque contra obstáculo fijo,True
2020-07-06 19:22:00,2020S009472,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True
2020-07-06 21:10:00,2020S009536,8.0,FUENCARRAL-EL PARDO,Atropello a persona,True


In [5]:
DataAcc = DataAcc.reset_index()

In [6]:
def Accidente1(tipo_accidente):
    global valor
    global fila  
    consulta = DataAcc.loc[fila]    
    distrito = consulta['distrito']
    
    if (distrito == 'FUENCARRAL-EL PARDO'):
        if (tipo_accidente == 'Colisión múltiple'): 
            valor = 1
        elif (tipo_accidente == 'Choque contra obstáculo fijo'):
            valor = 2
        elif (tipo_accidente == 'Colisión frontal') | (tipo_accidente == 'Colisión lateral') | (tipo_accidente == 'Colisión fronto-lateral'):
            valor = 3
        elif (tipo_accidente == 'Alcance'):
            valor = 4
        elif (tipo_accidente == 'Atropello a persona'):
            valor = 5
        elif (tipo_accidente == 'Atropello a animal'):
            valor = 6
    elif (distrito == 'HORTALEZA'):
        if (tipo_accidente == 'Colisión múltiple'): 
            valor = 7
        elif (tipo_accidente == 'Choque contra obstáculo fijo'):
            valor = 8
        elif (tipo_accidente == 'Colisión frontal') | (tipo_accidente == 'Colisión lateral') | (tipo_accidente == 'Colisión fronto-lateral'):
            valor = 9
        elif (tipo_accidente == 'Alcance'):
            valor = 10
        elif (tipo_accidente == 'Atropello a persona'):
            valor = 11
        elif (tipo_accidente == 'Atropello a animal'):
            valor = 12
    fila += 1 
    
    return (valor)

In [7]:
#Patrón Pardo: Hora de accidente es una hora antes y afEcta la hora siguiente si y solo si el minuto de la hora resultante es >=10
#Patrón Hortaleza: Accidente tiene efecto una hora después si y solo si el minuto de la hora es <10  y accidente por alcance no tienen ningún efecto
#Patrón Chamartín: La hora de accidente real es la hora registrada pero afecta a la hora siguiente.
def ModTime(ds):
    global valor
    global fila  
    date = pd.to_datetime(ds)
    hora = date.hour
    minuto = date.minute    
    dia = date.day
    mes = date.month    
    consulta = DataAcc.loc[fila]    
    distrito = consulta['distrito']        
    
    if (distrito == 'FUENCARRAL-EL PARDO'):
        try:
            if (hora > 0) & (minuto == 0):
                date = date.replace(hour=hora-1,minute=0)
            elif (hora == 0) & (minuto == 0):
                date = date.replace(day=dia-1,hour=23)
            elif (hora > 0) & (minuto < 10): ############REVISAR
                date = date.replace(hour=hora-1,minute=0)
            elif (hora == 0) & (minuto < 10):
                date = date.replace(day=dia-1,hour=23,minute=0)                
            else:
                date = date.replace(hour=hora,minute=0)
        except:
            res = calendar.monthrange(date.year, date.month)
            lday = res[1]
            date = date.replace(month=mes-1, day=lday, hour=23, minute=0)  
            
    elif (distrito == 'HORTALEZA'):
        try:
            if (hora < 23) & (minuto == 0):
                date = date.replace(hour=hora+1,minute=0)
            elif (hora == 23) & (minuto == 0):
                date = date.replace(day=dia+1,hour=0)
            elif (hora < 23) & (minuto < 10): ############REVISAR
                date = date.replace(hour=hora+1,minute=0)
            elif (hora == 23) & (minuto < 10):
                date = date.replace(day=dia+1,hour=0,minute=0)    
            elif (hora < 22) & (minuto >= 10):        
                date = date.replace(hour=hora+2,minute=0)         
            elif (hora == 22) & (minuto >= 10):
                date = date.replace(day=dia+1,hour=0,minute=0)
            elif (hora == 23) & (minuto >= 10):
                date = date.replace(day=dia+1,hour=1,minute=0)
        except:               
            if (hora == 23) & (minuto == 0):
                date = date.replace(month=mes+1,day=1,hour=0)            
            elif (hora == 23) & (minuto < 10):
                date = date.replace(month=mes+1,day=1,hour=0,minute=0)                                
            elif (hora == 22) & (minuto >= 10):
                date = date.replace(month=mes+1,day=1,hour=0,minute=0)
            elif (hora == 23) & (minuto >= 10):
                date = date.replace(month=mes+1,day=1,hour=1,minute=0)              
                
    elif (distrito == 'CHAMARTÍN'):
        try:
            if (hora < 23) & (minuto == 0):
                date = date.replace(hour=hora+1,minute=0)
            elif (hora == 23) & (minuto == 0):
                date = date.replace(day=dia+1,hour=0)
            elif (hora < 23) & (minuto <= 50):             
                date = date.replace(hour=hora+1,minute=0)            
            elif (hora == 23) & (minuto <= 50):
                date = date.replace(day=dia+1,hour=0,minute=0)                
            elif (hora < 22) & (minuto > 50):        
                date = date.replace(hour=hora+2,minute=0)         
            elif (hora == 22) & (minuto > 50):
                date = date.replace(day=dia+1,hour=0,minute=0)
            elif (hora == 23) & (minuto > 50):
                date = date.replace(day=dia+1,hour=1,minute=0)            
        except:
            if (hora == 23) & (minuto == 0):
                date = date.replace(month=mes+1,day=1,hour=0)            
            elif (hora == 23) & (minuto <= 50):
                date = date.replace(month=mes+1,day=1,hour=0,minute=0)                                
            elif (hora == 22) & (minuto > 50):
                date = date.replace(month=mes+1,day=1,hour=0,minute=0)
            elif (hora == 23) & (minuto > 50):
                date = date.replace(month=mes+1,day=1,hour=1,minute=0)                         
    
    return (date)
fila = 0
DataAcc['ds1'] = DataAcc['ds'].apply(ModTime)
DataAcc = DataAcc.drop_duplicates(subset = ['ds']) # Contabilizar en una nueva columna las repeticiones del index

In [8]:
DataAcc = DataAcc.set_index('ds')

In [9]:
Prueba = DataAcc.loc['2020-07-06':'2020-07-06']
Prueba

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente,ds1
ds,,,,,,
2020-07-06 12:00:00,2020S009461,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True,2020-07-06 13:00:00
2020-07-06 12:08:00,2020S009507,16.0,HORTALEZA,Colisión fronto-lateral,True,2020-07-06 13:00:00
2020-07-06 17:05:00,2020S009427,8.0,FUENCARRAL-EL PARDO,Choque contra obstáculo fijo,True,2020-07-06 18:00:00
2020-07-06 19:22:00,2020S009472,5.0,CHAMARTÍN,Choque contra obstáculo fijo,True,2020-07-06 21:00:00
2020-07-06 21:10:00,2020S009536,8.0,FUENCARRAL-EL PARDO,Atropello a persona,True,2020-07-06 23:00:00


In [10]:
# DataAcc = DataAcc.set_index('ds')
# Acc_ds = DataAcc.loc[:,['Accidente']]
# DataAcc = DataAcc.set_index('ds1')
# Acc_ds1 = DataAcc.loc[:,['Accidente1']]
# DataAcc = DataAcc.set_index('ds2')
# Acc_ds2 = DataAcc.loc[:,['Accidente2']]

# Acc_ds1 = Acc_ds1.reset_index() 
# Acc_ds1 = Acc_ds1.drop_duplicates(subset = ['ds1'])
# Acc_ds1 = Acc_ds1.set_index('ds1') 

In [11]:
# Acc_ds1
# Data=Acc_ds1.loc['2020-08-24':'2020-08-24']
# Data

In [12]:
# Acc_ds.to_csv("Acc2_ds.csv")
# Acc_ds1.to_csv("Acc2_ds1.csv")
# Acc_ds2.to_csv("Acc2_ds2.csv")